In [1]:
from keras.models import load_model
from keras.layers import Dense, Dropout, Conv1D, Input,MaxPooling1D,Flatten,LeakyReLU,AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
import random
import pandas as pd 
import numpy as np
from Bio import SeqIO
from keras import regularizers
from keras.metrics import binary_accuracy
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc,precision_recall_curve
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler
import os, sys, copy, getopt, re, argparse
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from keras import losses
import pickle

from scipy import interp
from sklearn.utils.class_weight import compute_class_weight

from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
import keras
from keras.initializers import RandomUniform
import keras.backend as K
from random import shuffle
import itertools 

Using TensorFlow backend.


In [4]:
def analyze(temp, OutputDir):

    trainning_result, validation_result, testing_result = temp

    file = open(OutputDir + '/performance.txt', 'w')

    index = 0
    for x in [trainning_result, validation_result, testing_result]:


        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        index += 1;

        file.write(title +  'results\n')


        for j in ['sn', 'sp', 'acc', 'MCC','AUC', 'precision', 'F1', 'lossValue']: 

            total = []

            for val in x:
                total.append(val[j])
            file.write(j + ' : mean : ' + str(np.mean(total)) + ' std : ' + str(np.std(total))  + '\n')

        file.write('\n\n______________________________\n')
    file.close()

    index = 0

    for x in [trainning_result, validation_result, testing_result]:

        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        
        i = 0

        for val in x:
            tpr = val['tpr']
            fpr = val['fpr']
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i+1, roc_auc))

            i += 1


        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        plt.savefig( OutputDir + '/' + title +'ROC.png')
        plt.close('all');
        
       #************************** Precision Recall Curve*********************************
        i = 0
        prs = []
        pre_aucs = []
        mean_recal= np.linspace(0, 1, 100)
        for val in x:
            pre = val['prec']
            rec = val['reca']
            prs.append(interp(mean_recal, rec, pre))
            prs[-1][0] = 0.0
            p_r_auc = auc(rec, pre)
            pre_aucs.append(p_r_auc)
            plt.plot(rec, pre, lw=1, alpha=0.3,label='PRC fold %d (AUC = %0.2f)' % (i+1, p_r_auc))

            i += 1


        mean_pre = np.mean(prs, axis=0)
        mean_auc = auc(mean_recal, mean_pre)
        std_auc = np.std(pre_aucs)
        plt.plot(mean_recal, mean_pre, color='b',
                 label=r'Mean PRC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_pre = np.std(prs, axis=0)
        pre_upper = np.minimum(mean_pre + std_pre, 1)
        pre_lower = np.maximum(mean_pre - std_pre, 0)
        plt.fill_between(mean_recal, pre_lower, pre_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision Recall curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        plt.savefig( OutputDir + '/' + title +'Pre_R_C.png')
        plt.close('all')


        index += 1

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out


#nucleotide chemical properties and frequency encoding
def cal(c,cb,i):
    bases ={'A':[1,1,1], 'C':[0,1,0], 'G':[1,0,0,], 'T':[0,0,1]}
    p=[]
    p=bases[c]
    p.append(np.round(cb/float(i+1),2))
    return(p)
def calculate(s):
    p=f=list()
    cba=cbc=cbt=cbg=0
    for i,c in enumerate(s):
        if c=='A':
            cba+=1
            p=cal(c,cba,i)
        elif c=='T':
            cbt+=1
            p=cal(c,cbt,i)
        elif c=='C':
            cbc+=1
            p=cal(c,cbc,i)
        elif c=='G':
            cbg+=1
            p=cal(c,cbg,i)
        else:
            p=[0,0,0,0]
        f.append(p)
    return(f)

def dataProcessing(seq,key):
    X_chem = []
    for a in range(len(seq)):
        X_chem.append(calculate(seq[a]))
    X_chem = np.array(X_chem)
    #print(X_chem.shape)

                

    if key == 1:
        lbs = list(np.ones(len(X_chem)))
    if key == 2:
        lbs=list(np.zeros(len(X_chem)))
    y = np.array(lbs, dtype = np.int32)
 
    return X_chem, y

def prepareData(path):
    all_seq = []
    for seq_record in SeqIO.parse(path, "fasta"):
        all_seq.append(str(seq_record.seq))
    pos_seq = []
    neg_seq = []
    for i in range(len(all_seq)):
        if(i < (len(all_seq)/2)):
            pos_seq.append(all_seq[i])
        else:
            neg_seq.append(all_seq[i])
    a=1
    b=2
    
    Positive_X, Positive_y = dataProcessing(pos_seq,a)
    Negitive_X, Negitive_y = dataProcessing(neg_seq,b)

    return Positive_X, Positive_y, Negitive_X, Negitive_y

def shuffleData(X, y):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    return X, y

def getMode():

    input_shape = (41,4)

    inputs = Input(shape = input_shape)

    convLayer = Conv1D(filters = 8, kernel_size = 3,padding='same',kernel_regularizer = regularizers.l2(1e-3),bias_regularizer = regularizers.l2(1e-4),activation = 'relu',input_shape = input_shape)(inputs);
    normalizationLayer=BatchNormalization()(convLayer)
    convLayer2 = Conv1D(filters = 16, kernel_size = 3,padding='same',kernel_regularizer = regularizers.l2(1e-3),bias_regularizer = regularizers.l2(1e-4),activation = 'relu',input_shape = input_shape)(normalizationLayer)
    flattenLayer = Flatten()(convLayer2)
    denseLayer = Dense(16, activation = 'relu',kernel_regularizer = regularizers.l2(1e-3),bias_regularizer = regularizers.l2(1e-4))(flattenLayer)
    dropoutLayer = Dropout(0.3)(denseLayer)
    outLayer = Dense(1, activation='sigmoid')(dropoutLayer)
    model = Model(inputs = inputs, outputs = outLayer)
    model.compile(loss='binary_crossentropy', optimizer= Adam(lr=0.001), metrics=[binary_accuracy]); #SGD(lr = 0.005, momentum=0.95)
    print(model.summary())
    return model


def calculateScore(X, y, model):
    
    score = model.evaluate(X,y)
    pred_y = model.predict(X)

    accuracy = score[1];

    tempLabel = np.zeros(shape = y.shape, dtype=np.int32)

    for i in range(len(y)):
        if pred_y[i] < 0.5:
            tempLabel[i] = 0
        else:
            tempLabel[i] = 1
    confusion = confusion_matrix(y, tempLabel)
    TN, FP, FN, TP = confusion.ravel()

    sensitivity = recall_score(y, tempLabel)
    specificity = TN / float(TN+FP)
    MCC = matthews_corrcoef(y, tempLabel)

    F1Score = (2 * TP) / float(2 * TP + FP + FN)
    precision = TP / float(TP + FP)
    recall = TP/float (TP+FN)
    pred_y = pred_y.reshape((-1, ))

    ROCArea = roc_auc_score(y, pred_y)
    fpr, tpr, thresholds = roc_curve(y, pred_y)
    lossValue = None
    
    pre, rec, threshlds = precision_recall_curve(y, pred_y)
    pre = np.fliplr([pre])[0] 
    rec = np.fliplr([rec])[0]  
    AUC_prec_rec = np.trapz(rec,pre)
    AUC_prec_rec = abs(AUC_prec_rec)
    
    print(y.shape)
    print(pred_y.shape)

    y_true = tf.convert_to_tensor(y, np.float32)
    y_pred = tf.convert_to_tensor(pred_y, np.float32)

    with tf.Session():
        lossValue = losses.binary_crossentropy(y_true, y_pred).eval()  
    plt.show()
    return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea,'precision' : precision, 'F1' : F1Score, 'fpr' : fpr, 'tpr' : tpr, 'thresholds' : thresholds, 'lossValue' : lossValue,'pre_recall_curve':AUC_prec_rec,'prec':pre,'reca':rec}
def test_data_prepro(ind_test):
    Positive_X, Positive_y, Negitive_X, Negitive_y = prepareData(ind_test)
    test_X = np.concatenate((Positive_X,Negitive_X))
    test_y = np.concatenate((Positive_y,Negitive_y))
    return test_X, test_y
def funciton(All_data, OutputDir, folds):

    Positive_X, Positive_y, Negitive_X, Negitive_y = prepareData(All_data)
    random.shuffle(Positive_X);
    random.shuffle(Negitive_X);
    Positive_X_Slices = chunkIt(Positive_X, folds)
    Positive_y_Slices = chunkIt(Positive_y, folds)
    Negative_X_Slices = chunkIt(Negitive_X, folds)
    Negative_y_Slices = chunkIt(Negitive_y, folds)

    trainning_result = []
    validation_result = []
    testing_result = []
    for test_index in range(folds):

        test_X = np.concatenate((Positive_X_Slices[test_index],Negative_X_Slices[test_index]))
        test_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))
        
        validation_index = (test_index+1) % folds;

        valid_X = np.concatenate((Positive_X_Slices[validation_index],Negative_X_Slices[validation_index]))
        valid_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))

        start = 0

        for val in range(0, folds):
            if val != test_index and val != validation_index:
                start = val
                break

        train_X = np.concatenate((Positive_X_Slices[start],Negative_X_Slices[start]))
        train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

        for i in range(0, folds):
            if i != test_index and i != validation_index and i != start:
                tempX = np.concatenate((Positive_X_Slices[i],Negative_X_Slices[i]))
                tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))

                
                train_X = np.concatenate((train_X, tempX))
                train_y = np.concatenate((train_y, tempy))

    
        test_X, test_y = shuffleData(test_X,test_y)
        valid_X,valid_y = shuffleData(valid_X,valid_y)
        train_X,train_y = shuffleData(train_X,train_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'x_test',test_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'y_test',test_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'valid_X',valid_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'valid_y',valid_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'x_train',train_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'y_train',train_y)
        
        
        model = getMode()
        
        early_stopping = EarlyStopping(monitor='val_loss', patience= 10, restore_best_weights=True)
        model_check = ModelCheckpoint(filepath = OutputDir + "/model" + str(test_index+1) +".h5",mode='min', monitor = 'val_loss', save_best_only=True)#, save_weights_only=True
        reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.01, patience=10)
        cbacks = [model_check,reduct_L_rate,early_stopping]
        
        history = model.fit(train_X, train_y, batch_size = 16, epochs =200,validation_data = (valid_X, valid_y),callbacks = cbacks);
        model=load_model('C:/Users/NSCL/N4-methylctosine//outputs//model'+str(test_index+1)+'.h5')
        
        trainning_result.append(calculateScore(train_X, train_y, model))
        validation_result.append(calculateScore(valid_X, valid_y, model))
        testing_result.append(calculateScore(test_X, test_y, model))

    temp_dict = (trainning_result, validation_result, testing_result)
    analyze(temp_dict, OutputDir)

In [5]:
All_data = 'G.subterraneus.fasta.txt' 
OutputDir = 'C:/Users/NSCL/N4-methylctosine//outputs/'
funciton(All_data, OutputDir, 10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 41, 4)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 41, 8)             104       
_________________________________________________________________
batch_normalization_2 (Batch (None, 41, 8)             32        
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 41, 16)            400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 656)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                10512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
__________

180/180 [==============================] - 0s 83us/step
(180,)
(180,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 41, 4)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 41, 8)             104       
_________________________________________________________________
batch_normalization_3 (Batch (None, 41, 8)             32        
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 41, 16)            400       
_________________________________________________________________
flatten_3 (Flatten)          (None, 656)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                10512     
_________________________________________________________________
dropou

Train on 1448 samples, validate on 182 samples
Epoch 1/200
1448/1448 [==============================] - 2s 1ms/step - loss: 0.6546 - binary_accuracy: 0.6692 - val_loss: 0.5124 - val_binary_accuracy: 0.7967
Epoch 2/200
1448/1448 [==============================] - 0s 209us/step - loss: 0.4751 - binary_accuracy: 0.8080 - val_loss: 0.3733 - val_binary_accuracy: 0.8516
Epoch 3/200
1448/1448 [==============================] - 0s 200us/step - loss: 0.3797 - binary_accuracy: 0.8715 - val_loss: 0.4227 - val_binary_accuracy: 0.8516
Epoch 4/200
1448/1448 [==============================] - 0s 208us/step - loss: 0.3361 - binary_accuracy: 0.8950 - val_loss: 0.2751 - val_binary_accuracy: 0.9066
Epoch 5/200
1448/1448 [==============================] - 0s 222us/step - loss: 0.2841 - binary_accuracy: 0.9192 - val_loss: 0.2530 - val_binary_accuracy: 0.9341
Epoch 6/200
1448/1448 [==============================] - 0s 195us/step - loss: 0.2451 - binary_accuracy: 0.9309 - val_loss: 0.2463 - val_binary_accura

1448/1448 [==============================] - 0s 192us/step - loss: 0.3031 - binary_accuracy: 0.8999 - val_loss: 0.2473 - val_binary_accuracy: 0.9444
Epoch 5/200
1448/1448 [==============================] - 0s 179us/step - loss: 0.2634 - binary_accuracy: 0.9247 - val_loss: 0.2238 - val_binary_accuracy: 0.9444
Epoch 6/200
1448/1448 [==============================] - 0s 186us/step - loss: 0.2271 - binary_accuracy: 0.9316 - val_loss: 0.1858 - val_binary_accuracy: 0.9500
Epoch 7/200
1448/1448 [==============================] - 0s 185us/step - loss: 0.1968 - binary_accuracy: 0.9448 - val_loss: 0.1790 - val_binary_accuracy: 0.9611
Epoch 8/200
1448/1448 [==============================] - 0s 197us/step - loss: 0.1803 - binary_accuracy: 0.9634 - val_loss: 0.1628 - val_binary_accuracy: 0.9556
Epoch 9/200
1448/1448 [==============================] - 0s 197us/step - loss: 0.1669 - binary_accuracy: 0.9620 - val_loss: 0.1756 - val_binary_accuracy: 0.9611
Epoch 10/200
1448/1448 [======================

Train on 1448 samples, validate on 182 samples
Epoch 1/200
1448/1448 [==============================] - 2s 2ms/step - loss: 0.6099 - binary_accuracy: 0.6920 - val_loss: 0.4467 - val_binary_accuracy: 0.8352
Epoch 2/200
1448/1448 [==============================] - 0s 200us/step - loss: 0.4096 - binary_accuracy: 0.8453 - val_loss: 0.3803 - val_binary_accuracy: 0.8626
Epoch 3/200
1448/1448 [==============================] - 0s 196us/step - loss: 0.3407 - binary_accuracy: 0.8847 - val_loss: 0.3271 - val_binary_accuracy: 0.8736
Epoch 4/200
1448/1448 [==============================] - 0s 253us/step - loss: 0.2938 - binary_accuracy: 0.9061 - val_loss: 0.3031 - val_binary_accuracy: 0.8901
Epoch 5/200
1448/1448 [==============================] - 0s 214us/step - loss: 0.2506 - binary_accuracy: 0.9247 - val_loss: 0.2898 - val_binary_accuracy: 0.9121
Epoch 6/200
1448/1448 [==============================] - 0s 216us/step - loss: 0.2228 - binary_accuracy: 0.9427 - val_loss: 0.2842 - val_binary_accura

182/182 [==============================] - 0s 71us/step
(182,)
(182,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 41, 4)             0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 41, 8)             104       
_________________________________________________________________
batch_normalization_8 (Batch (None, 41, 8)             32        
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 41, 16)            400       
_________________________________________________________________
flatten_8 (Flatten)          (None, 656)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 16)                10512     
_________________________________________________________________
dropou

1448/1448 [==============================] - 0s 223us/step - loss: 0.0916 - binary_accuracy: 0.9876 - val_loss: 0.4828 - val_binary_accuracy: 0.8333
Epoch 16/200
1448/1448 [==============================] - 1s 511us/step
(1448,)
(1448,)
180/180 [==============================] - 0s 194us/step
(180,)
(180,)
182/182 [==============================] - 0s 55us/step
(182,)
(182,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 41, 4)             0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 41, 8)             104       
_________________________________________________________________
batch_normalization_10 (Batc (None, 41, 8)             32        
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 41, 16)            400       
____________________________

1448/1448 [==============================] - 0s 282us/step - loss: 0.0986 - binary_accuracy: 0.9848 - val_loss: 0.0975 - val_binary_accuracy: 0.9944
Epoch 18/200
1448/1448 [==============================] - 0s 260us/step - loss: 0.0813 - binary_accuracy: 0.9965 - val_loss: 0.0808 - val_binary_accuracy: 0.9944
Epoch 19/200
1448/1448 [==============================] - 0s 281us/step - loss: 0.0833 - binary_accuracy: 0.9931 - val_loss: 0.0839 - val_binary_accuracy: 0.9944
Epoch 20/200
1448/1448 [==============================] - 0s 341us/step - loss: 0.0799 - binary_accuracy: 0.9952 - val_loss: 0.0900 - val_binary_accuracy: 0.9944
Epoch 21/200
1448/1448 [==============================] - 0s 296us/step - loss: 0.0813 - binary_accuracy: 0.9910 - val_loss: 0.0835 - val_binary_accuracy: 0.9944
Epoch 22/200
1448/1448 [==============================] - 0s 276us/step - loss: 0.0857 - binary_accuracy: 0.9876 - val_loss: 0.0950 - val_binary_accuracy: 0.9944
Epoch 23/200
1448/1448 [=================